In [1]:
import numpy as np
import faiss
import os
import jieba
import copy
from FlagEmbedding import FlagModel
import json

/home/lsh/anaconda3/envs/DF-retrieval/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!curl -I https://huggingface.co

HTTP/1.1 200 Connection established

HTTP/2 200 
content-type: text/html; charset=utf-8
content-length: 111672
date: Thu, 10 Oct 2024 16:56:16 GMT
x-powered-by: huggingface-moon
cross-origin-opener-policy: same-origin
referrer-policy: strict-origin-when-cross-origin
x-request-id: Root=1-67080730-468ef3a87279554f6424ac55
x-frame-options: DENY
etag: W/"1b438-m+8hbM7IOInl7tGbrUC3jCAvuh0"
vary: Accept-Encoding
x-cache: Hit from cloudfront
via: 1.1 837e503aed9da880c3776b678e912f88.cloudfront.net (CloudFront)
x-amz-cf-pop: HKG1-P1
x-amz-cf-id: Le9H4-2xRWFFl0k5HXzW3AftYmsnXcrY2OHwEiEbU8MSgEgnhZNRzQ==
age: 47



In [ ]:
# %env ALL_PROXY=http://localhost:7890
# %env HTTP_PROXY=http://localhost:7890
# %env HTTPS_PROXY=http://localhost:7890

In [5]:

!curl -I https://huggingface.co
model = FlagModel('BAAI/bge-large-zh-v1.5',
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=True)

HTTP/1.1 200 Connection established

HTTP/2 200 
content-type: text/html; charset=utf-8
content-length: 111675
date: Thu, 10 Oct 2024 18:31:31 GMT
x-powered-by: huggingface-moon
cross-origin-opener-policy: same-origin
referrer-policy: strict-origin-when-cross-origin
x-request-id: Root=1-67081d83-39d2656d26a2eabd16d3cc85
x-frame-options: DENY
etag: W/"1b43b-VhqMxFlsnyByFCP9hiyHfSQF9Pw"
vary: Accept-Encoding
x-cache: Hit from cloudfront
via: 1.1 7f3ca3166f303ad5837916b99ebc5436.cloudfront.net (CloudFront)
x-amz-cf-pop: HKG1-P1
x-amz-cf-id: 5l93kNOrDhMs1sxwnd1qyIhGHhkymS03VXMx5Cc1oidM-XrVMhr_VA==
age: 23



In [3]:
import torch
torch.__version__
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.4


In [11]:
# with open('resources/train-v2.0.json', 'r') as file:
#     squad_data = json.load(file)

folder_path = '/home/lsh/DM-DataFountain-Retrieval/resources/temp'
output_path = '/home/lsh/DM-DataFountain-Retrieval/resources'

# 遍历文件夹及其子文件夹中的所有文件
vectors = []
datas = []

file_num = 1
text_num = 1

for item in os.listdir(folder_path):
    file_path = os.path.join(folder_path, item)
    if not os.path.isfile(full_path):
        continue
    with open(file_path, 'r', encoding='utf-8') as f:
        text_num = 1
        data = json.load(f)
        print(file_path)
        for text in data:
            if isinstance(text, str):
                vec = model.encode(text)
                vectors.append(vec)
                datas.append({'content': text, 'vectors': vec.tolist()})
            else:
                print('wrong:', end='')
                print(text)
        print('file:' + str(file_num) + 'complete')
        file_num += 1
            
with open(output_path + '/vectors', 'w') as file:
    json.dump(datas, file, ensure_ascii=False, indent=4)

/home/lsh/DM-DataFountain-Retrieval/resources/temp/91.json
file:1complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/17.json
file:2complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/4.json
file:3complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/18.json
file:4complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/24.json
file:5complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/118.json
file:6complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/76.json
file:7complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/28.json
file:8complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/60.json
file:9complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/6.json
file:10complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/12.json
file:11complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/106.json
file:12complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/43.json
file:13complete
/home/lsh/DM-DataFountain-Retrieva

TypeError: Object of type ndarray is not JSON serializable

In [ ]:
for item in datas:
    item['vectors'] = item['vectors'].tolist()

with open(output_path + '/vectors', 'w') as file:
    json.dump(datas, file, ensure_ascii=False, indent=4)

In [62]:
word_to_index = {word: index for index, word in enumerate(vocabulary)}

In [63]:
with open(output_path + '/word_index', 'w') as file:
    for word, index in word_to_index.items():
        file.write(word + ':' + str(index) + '\n')

In [64]:
def convert_text_to_vector(text):
    words = jieba.lcut(text)
    bow_vector = np.zeros(len(vocabulary))
    for word in words:
        if word in word_to_index:
            bow_vector[word_to_index[word]] = 1    
    return bow_vector


paragraph_vectors = []
paragraphs = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()  # 读取文件内容
            paragraphs.append(copy.deepcopy(content))
            paragraph_vectors.append(convert_text_to_vector(content))

In [66]:
with open(output_path + '/paragraph_vectors', 'w') as file:
    for text in paragraph_vectors:
        for num in text:
            file.write(str(num))
        file.write('\n')

In [67]:
dimension = len(vocabulary)
index = faiss.IndexFlatL2(dimension)
# 将 NumPy 数组转换为 1 个二维数组
paragraph_vectors = np.stack(paragraph_vectors).astype('float32')
index.add(paragraph_vectors)
print(index)



<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7e1c25f944b0> >


In [71]:
def search_for_paragraphs(search_term, num_results, index):
    search_vector = convert_text_to_vector(search_term)
    search_vector = np.array([search_vector]).astype('float32')
    distances, indexes = index.search(search_vector, num_results)
    for i, (distance, index) in enumerate(zip(distances[0], indexes[0])):
        print(f"Result {i+1}, Distance: {distance}")
        print(paragraphs[index])
        print()


In [73]:
search_for_paragraphs('根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？', 5, index)

Result 1, Distance: 301.0
本文档为2024CCFBDCI比赛用语料的一部分。部分文档使用大语言模型改写生成，内容可能与现实情况 不符，可能不具备现实意义，仅允许在本次比赛中使用。 中国联通成为北京2022年冬奥会和冬残奥会官方通信服务合作伙伴 发布时间：2017-12-26发布人：新闻发布人 12月26日晚间，在中国联通大厦里，中国联通的红色“中国结”与北京2022年冬奥会会徽和冬残奥会会徽的组合标志格外 醒目，中国联通与北京2022年冬奥会和冬残奥会组织委员会签约仪式在这里举行。这场签约，标志着中国联通正式成为北京 2022年冬奥会和冬残奥会官方通信服务合作伙伴，跻身于北京冬奥组委市场开发计划最高级别的赞助企业之列。 当晚，北京市副市长、北京冬奥组委执行副主席张建东，中国残联理事长、北京冬奥组委副主席鲁勇；中国联通董事长王 晓初，中国联通总经理陆益民，中国联通副总经理邵广禄出席签约仪式。北京冬奥组委秘书长韩子荣与中国联通副总经理买彦 州代表双方签署赞助协议。 北京市副市长、北京冬奥组委执行副主席张建东致辞表示，北京冬奥会是我国重要历史节点的重大标志性活动，对于展现 新时代国家形象、促进国家发展、振奋民族精神，具有重大而深远的意义。与通信服务企业开展合作，不仅是实施市场开发计 划的重要内容，也是高质量、高效率推进筹办工作的迫切需要。中国联通是我国电信事业的领军企业，拥有庞大的通信服务资 源和丰富的重大活动服务保障经验。北京冬奥组委将携手中国联通加强北京冬奥会与国内外各界的相联相通，努力打造智慧冬 奥、科技冬奥；同时，也会努力搭建推广平台，提供良好服务，使中国联通充分获得奥运品牌带来的广泛收益，助力企业更好 发展。 中国联通董事长王晓初在致辞中表示，近年来，在“网络强国”“数字中国”“互联网+行动计划”等国家战略的引领下，中国联 通坚持新发展理念，全面实施以“聚焦、创新、合作”为主要内涵的企业新战略。我们把致力于成为“客户信赖的智慧生活创造者” 作为企业愿景，把“联通世界 创享美好智慧生活”作为企业使命。奥运会的通信保障，联通着奥运健儿和热情观众，联通着好客 中国与广阔世界，联通着伟大复兴中国梦与现代奥运精神。中国联通将围绕“智慧冬奥、联通未来”这一主题，以智慧的网络、极 致的速率支撑奥运，以智慧的应用、丰富的产品服务奥运，以智慧的技术、专业